### Proudly presented by team mucki-schnucki

Import of packages

In [64]:
import pandas as pd
import numpy as np

Import of data

In [65]:
df_Weather = pd.read_csv('weather_hourly_boston.csv')

Now wrong values of the duration are excluded of the data frame

In [105]:
df_Trips_Modified = df_Trips[(df_Trips['duration'].dt.total_seconds() > 0)] & df_Trips[(df_Trips['duration'].dt.total_seconds() < 18000)]

,start_time,end_time,start_station_id,end_station_id,start_station_name,end_station_name,bike_id,user_type,start_hour,weekday,duration
1175230,2017-11-05 01:52:32,2017-11-05 01:00:38,55,178,Boylston St at Massachusetts Ave,MIT Pacific St at Purrington St,1038,Subscriber,1,6,-1 days +23:08:06
1175233,2017-11-05 01:55:32,2017-11-05 01:06:34,48,39,Post Office Square - Pearl St at Milk St,Washington St at Rutland St,1916,Subscriber,1,6,-1 days +23:11:02
1175235,2017-11-05 01:57:53,2017-11-05 01:08:30,90,80,Lechmere Station at Cambridge St / First St,MIT Stata Center at Vassar St / Main St,1021,Subscriber,1,6,-1 days +23:10:37
1175236,2017-11-05 01:58:28,2017-11-05 01:16:59,49,178,Stuart St at Charles St,MIT Pacific St at Purrington St,1139,Subscriber,1,6,-1 days +23:18:31
1175237,2017-11-05 01:58:50,2017-11-05 01:04:57,49,60,Stuart St at Charles St,Charles Circle - Charles St at Cambridge St,847,Subscriber,1,6,-1 days +23:06:07
1175238,2017-11-05 01:59:59,2017-11-05 01:21:46,49,130,Stuart St at Charles St,Upham's Corner,1645,Subscriber,1,6,-1 days +23:21:47


### Weather data Cleaning 

In [80]:
df_Weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43848 entries, 0 to 43847
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date_time  43354 non-null  object 
 1   max_temp   43354 non-null  float64
 2   min_temp   43354 non-null  float64
 3   precip     43356 non-null  float64
dtypes: float64(3), object(1)
memory usage: 1.3+ MB


In [81]:
df_Weather.isnull().values.sum()

1974

In [82]:
df_Weather.head(5)

,date_time,max_temp,min_temp,precip
0,2015-01-02 01:00:00,-1.1,-1.1,0.0
1,2015-01-02 02:00:00,-1.1,-1.1,0.0
2,2015-01-02 03:00:00,-0.6,-0.6,0.0
3,2015-01-02 04:00:00,-0.6,-0.6,0.0
4,2015-01-02 05:00:00,-0.6,-0.6,0.0


In [83]:
df_Weather.tail(5)

,date_time,max_temp,min_temp,precip
43843,2020-01-01 20:00:00,5.0,5.0,0.0
43844,2020-01-01 21:00:00,4.4,4.4,0.0
43845,2020-01-01 22:00:00,4.4,4.4,0.0
43846,2020-01-01 23:00:00,3.9,3.9,0.0
43847,2020-01-02 00:00:00,3.3,3.3,0.0


The dataset containing information about the weather reaches from 2015 - 2020. As only data of 2017 is needed, the data can limited to this year. 

In [90]:
df_Weather['date_time'] = pd.to_datetime(df_Weather['date_time'])

df_Weather2 = df_Weather[(df_Weather['date_time'] > "2017-01-01") & (df_Weather['date_time'] < "2018-01-01")]


In [85]:
df_Weather2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8689 entries, 17520 to 26302
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date_time  8689 non-null   datetime64[ns]
 1   max_temp   8689 non-null   float64       
 2   min_temp   8689 non-null   float64       
 3   precip     8689 non-null   float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 339.4 KB


To display information about every hour in 2017 the weather data should contain 365 * 24 = 8760 rows. As the filtered dataframe only has 8689 entries, information about 71 hours is missing. 

In [86]:
df_Weather2.isnull().values.sum()

0

Fortunatley no NA values are in the dataset anymore.

In [87]:
df_Weather2.head(5)

,date_time,max_temp,min_temp,precip
17520,2017-01-01 01:00:00,4.4,4.4,0.0
17521,2017-01-01 02:00:00,5.0,5.0,1.0
17522,2017-01-01 03:00:00,5.0,5.0,1.0
17523,2017-01-01 04:00:00,5.0,4.4,1.0
17524,2017-01-01 05:00:00,4.4,4.4,1.0


In [88]:
df_Weather2.tail(5)

,date_time,max_temp,min_temp,precip
26298,2017-12-31 19:00:00,-11.1,-11.1,0.0
26299,2017-12-31 20:00:00,-10.6,-10.6,0.0
26300,2017-12-31 21:00:00,-11.1,-11.1,0.0
26301,2017-12-31 22:00:00,-11.7,-11.7,0.0
26302,2017-12-31 23:00:00,-11.1,-11.1,0.0


In [89]:
df_Stations.head()

,Number,Name,Latitude,Longitude,District,Public,Total docks
0,K32015,1200 Beacon St,42.344149,-71.114674,Brookline,Yes,15
1,W32006,160 Arsenal,42.364664,-71.175694,Watertown,Yes,11
2,A32019,175 N Harvard St,42.363796,-71.129164,Boston,Yes,18
3,S32035,191 Beacon St,42.380323,-71.108786,Somerville,Yes,19
4,C32094,2 Hummingbird Lane at Olmsted Green,42.288870,-71.095003,Boston,Yes,17
